In [1]:
from gerrychain import (GeographicPartition, Partition, Graph, proposals, updaters, constraints, accept, Election, grid)
from gerrychain.metrics import mean_median, partisan_bias, polsby_popper, efficiency_gap, partisan_gini
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import pickle
import json
import time
from functools import partial
from chain import MarkovChain as mc2
from plotting import *
import networkx as nx

In [2]:
run run_chains4

In [3]:
graph = Graph.from_json('graph_combined_vs_2018.json')

In [4]:
my_updaters = {"population": updaters.Tally('POP100', alias="population"),
                       "polsby_popper": polsby_popper,
                       "split_counties": SplitCounties(),
                       'SEN10': Election('SEN10', {"Dem": "SEN_DEM", "Rep": "SEN_REP"}),
                       'G10': Election('G10', {"Dem": "G_DEM", "Rep": "G_REP"}),
                       'COMB10': Election('COMB10', {"Dem":"COMB_DEM", "Rep":"COMB_REP"}),
                       'assignment_array': AssignmentArray()}

In [5]:
initial_partition = GeographicPartition(graph, 'US_Distric', my_updaters)

In [6]:
a = initial_partition.assignment

In [7]:
a2 = dict()
for key in a.keys():
    if a[key] in [0, 1, 2]:
        a2[key] = 0
    else:
        a2[key] = 1

In [8]:
ideal = np.mean(list(initial_partition['population'].values()))

In [11]:
custom = lambda p: 0.5*len(p['cut_edges']) + 200*(1 - p['population'][1]/ideal)**2
c = Chain('flip-mh', 10000, starting_assignment=a2, storage_ratio=100, weights={'custom':1}, custom=custom)

In [10]:
plot_district_map(c.stored_assignments.loc[99, :])

KeyError: 99

In [ ]:
c.chain.attempted

In [ ]:
c.data['POP2'].describe()

In [ ]:
c.data

In [ ]:
key = 'Sorted SenRep Vote Share 1'
plt.hist(c.data[key], bins=50)
plt.gca().axvline(x=c.data[key][0], color='r', lw=2)